<a href="https://colab.research.google.com/github/kirath2205/Final-Year-Project/blob/main/FGSM_attack_and_PGD_attack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras,os
import tensorflow as tf
from keras.models import Sequential,Model
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D , Dropout
import numpy as np
from tensorflow import keras
from keras.initializers import glorot_uniform
from keras import applications
from tensorflow.keras.applications.resnet50 import ResNet50 , preprocess_input
from keras import regularizers
from absl import app, flags

# Install bleeding edge version of cleverhans
!pip install git+https://github.com/tensorflow/cleverhans.git#egg=cleverhans
from cleverhans.tf2.attacks.projected_gradient_descent import projected_gradient_descent
from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method

  Cloning https://github.com/tensorflow/cleverhans.git to /tmp/pip-install-s3c1clxa/cleverhans_2416fda0692f4224b3b12b34c393c14f
  Running command git clone -q https://github.com/tensorflow/cleverhans.git /tmp/pip-install-s3c1clxa/cleverhans_2416fda0692f4224b3b12b34c393c14f
     |████████████████████████████████| 154 kB 32.2 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
  Created wheel for cleverhans: filename=cleverhans-4.0.0-py3-none-any.whl size=92423 sha256=ed513e38964f735528db7c04d799a85759290689e0b75fea46f285a642e11665
  Stored in directory: /tmp/pip-ephem-wheel-cache-4ojjjdfc/wheels/60/54/1e/97e3fe32d62bd252c9fbbee44a0545028c6018b81c054af3e4
Successfully built cleverhans


In [2]:
cnn_model = keras.models.load_model('cnn_model.h5')
#vgg_16 = keras.models.load_model('vgg16.h5')
#resnet = keras.models.load_model('resnet.h5')

In [3]:
cifar_10 = tf.keras.datasets.cifar10.load_data()
(X_train, y_train), (X_test, y_test) = cifar_10
def pre_processing_cifar10(X_train,y_train,X_test,y_test,image_channel):
  X_train = X_train.astype('float32')
  X_test = X_test.astype('float32')

  y_train = keras.utils.to_categorical(y_train)
  y_test = keras.utils.to_categorical(y_test)

  X_train = preprocess_input(X_train)
  X_test = preprocess_input(X_test)

  img_width = X_train[0].shape[0]
  img_height = X_train[0].shape[1]
  image_channel = image_channel

  input_shape=(img_width,img_height,image_channel) 
  return X_train,y_train,X_test,y_test
X_train,y_train,X_test,y_test = pre_processing_cifar10(X_train,y_train,X_test,y_test,3)

170508288/170498071 [==============================] - 2s 0us/step


In [15]:

def make_predictions(model , image , true_value):

  labels = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

  true_label_index = -1
  for i in range(len(true_value)):

    if(true_value[i]==1):
      true_label_index = i
      break

  prediction = model.predict(image)[0]
  probability = float('-inf')
  predicted_label_index = -1

  for i in range(len(prediction)):

    if(prediction[i]>probability):
      probability = prediction[i]
      predicted_label_index = i

  predicted_label = labels[predicted_label_index]
  true_label = labels[true_label_index]

  print('Predicted Label -',predicted_label,' Probability -',probability)
  print('True Label -',true_label)

  return predicted_label , probability 


In [ ]:
from keras.preprocessing.image import img_to_array

epsilon = 0.2
logits_cnn_model = tf.keras.Model(cnn_model.input,cnn_model.layers[-1].output)
total_images = 100
misclassifications_fgsm = []
misclassifications_pgd = []
FLAGS = flags.FLAGS

for image_index in range(total_images):
  
  print('-------------------------------')
  print()
  print('Image index :',image_index)
  print()
  print('Original Sample')
  image  = X_test[image_index]
  image = img_to_array(image)
  image = image.reshape(1, 32, 32, 3)
  true_value = y_test[image_index]
  original_prediction = make_predictions(cnn_model,image,true_value)
  fgsm_sample = fast_gradient_method(logits_cnn_model, image, epsilon, np.inf, targeted=False)
  pgd_sample = projected_gradient_descent(cnn_model, image, 0.2, 0.01, 40, np.inf)
  print()
  print('Untargeted FGSM Adverserial Sample')
  fgsm_prediction = make_predictions(cnn_model , fgsm_sample , true_value)
  print()
  print('PGD Adverserial Sample')
  pgd_prediction = make_predictions(cnn_model , pgd_sample , true_value)
  print()
  
  if(pgd_prediction[0]!=original_prediction[0]):
    original_model_label = original_prediction[0]
    pgd_label = pgd_prediction[0]
    original_probability = original_prediction[1]
    pgd_probability = pgd_prediction[1]
    misclassifications_pgd.append({'Original Model Label':original_model_label,'original probability':original_probability,'Prediction after PGD attack':pgd_label,'Probability after PGD attack':pgd_probability,'Image index':image_index})

  if(fgsm_prediction[0]!=original_prediction[0]):
    original_model_label = original_prediction[0]
    fgsm_label = fgsm_prediction[0]
    original_probability = original_prediction[1]
    fgsm_probability = fgsm_prediction[1]
    misclassifications_fgsm.append({'Original Model Label':original_model_label,'original probability':original_probability,'Prediction after FGSM attack':fgsm_label,'Probability after FGSM attack':fgsm_probability,'Image index':image_index})
     
  print('-------------------------------')


In [18]:
print('Mispredictions due to FGSM attack')
print()
print('-------------------------------')
for element in misclassifications_fgsm:
  print(element)

Mispredictions due to FGSM attack

-------------------------------
{'Original Model Label': 'airplane', 'original probability': 0.720281, 'Prediction after FGSM attack': 'ship', 'Probability after FGSM attack': 0.51244044, 'Image index': 3}
{'Original Model Label': 'automobile', 'original probability': 0.55166876, 'Prediction after FGSM attack': 'truck', 'Probability after FGSM attack': 0.7046554, 'Image index': 6}
{'Original Model Label': 'bird', 'original probability': 0.39542064, 'Prediction after FGSM attack': 'deer', 'Probability after FGSM attack': 0.33747023, 'Image index': 32}
{'Original Model Label': 'cat', 'original probability': 0.36319295, 'Prediction after FGSM attack': 'dog', 'Probability after FGSM attack': 0.2865367, 'Image index': 33}
{'Original Model Label': 'cat', 'original probability': 0.5012216, 'Prediction after FGSM attack': 'ship', 'Probability after FGSM attack': 0.4071542, 'Image index': 35}
{'Original Model Label': 'truck', 'original probability': 0.5209432,

In [19]:
print('Mispredictions due to PGD attack')
print()
print('-------------------------------')
for element in misclassifications_pgd:
  print(element)

Mispredictions due to PGD attack

-------------------------------
{'Original Model Label': 'airplane', 'original probability': 0.720281, 'Prediction after PGD attack': 'ship', 'Probability after PGD attack': 0.5262787, 'Image index': 3}
{'Original Model Label': 'automobile', 'original probability': 0.55166876, 'Prediction after PGD attack': 'truck', 'Probability after PGD attack': 0.7113933, 'Image index': 6}
{'Original Model Label': 'bird', 'original probability': 0.60332745, 'Prediction after PGD attack': 'cat', 'Probability after PGD attack': 0.43106443, 'Image index': 25}
{'Original Model Label': 'bird', 'original probability': 0.39542064, 'Prediction after PGD attack': 'deer', 'Probability after PGD attack': 0.3368668, 'Image index': 32}
{'Original Model Label': 'cat', 'original probability': 0.36319295, 'Prediction after PGD attack': 'dog', 'Probability after PGD attack': 0.28503263, 'Image index': 33}
{'Original Model Label': 'cat', 'original probability': 0.5012216, 'Prediction